<a href="https://colab.research.google.com/github/prc98/title-generation/blob/main/NLP_TITLE_GENERATION_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
import matplotlib as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler
from torch.optim import lr_scheduler
from PIL import Image
import timeit
from sklearn.pipeline import Pipeline
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy import data
import random
## For reproducibility
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
random.seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Create Preprocessing Pipeline

In [ ]:
tokenize =  lambda s: s.split()

In [ ]:
import re  
def cleanup_text(texts):
    cleaned_text = []
    for text in texts:
        # remove punctuation
        text = re.sub('[^a-zA-Z0-9]', ' ', text)
        # remove multiple spaces
        text = re.sub(r' +', ' ', text)
        # remove newline
        text = re.sub(r'\n', ' ', text)
        
        cleaned_text.append(text)
    return cleaned_text

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Create two torch text fields

In [ ]:
ABS = data.Field(tokenize = tokenize,init_token='<sos>',eos_token='<eos>',pad_first=True,lower = True,stop_words=stop,preprocessing=cleanup_text)
TITLE = data.Field(tokenize = tokenize,init_token='<sos>',eos_token='<eos>',lower = True,preprocessing=cleanup_text)

#### Read tabular dataset

In [ ]:
fields = [('Abstract',ABS),('Title',TITLE)]

In [ ]:
dataset = data.TabularDataset(path='./drive/MyDrive/data.csv',format='csv', fields=fields,skip_header=True)

In [ ]:
import random
train_data, valid_data = dataset.split(split_ratio=0.9, random_state=random.seed(0))

In [ ]:
print(len(train_data.examples))
print(len(valid_data.examples))

36900
4100


In [ ]:
train_data[0].Abstract

['human',
 'communication',
 'typically',
 'underlying',
 'structure ',
 'reflected',
 'fact',
 'many',
 'user',
 'generated',
 'videos ',
 'starting',
 'point ',
 'ending ',
 'certain',
 'objective',
 'steps',
 'two',
 'identified ',
 'paper ',
 'propose',
 'method',
 'parsing',
 'video',
 'semantic',
 'steps',
 'unsupervised',
 'way ',
 'proposed',
 'method',
 'capable',
 'providing',
 'semantic',
 ' storyline ',
 'video',
 'composed',
 'objective',
 'steps ',
 'accomplish',
 'using',
 'visual',
 'language',
 'cues',
 'joint',
 'generative',
 'model ',
 'proposed',
 'method',
 'also',
 'provide',
 'textual',
 'description',
 'identified',
 'semantic',
 'steps',
 'video',
 'segments ',
 'evaluate',
 'method',
 'large',
 'number',
 'complex',
 'youtube',
 'videos',
 'show',
 'results',
 'unprecedented',
 'quality',
 'intricate',
 'impactful',
 'problem ']

In [ ]:
for i in train_data.Abstract:
  print(i)
  break

['human', 'communication', 'typically', 'underlying', 'structure ', 'reflected', 'fact', 'many', 'user', 'generated', 'videos ', 'starting', 'point ', 'ending ', 'certain', 'objective', 'steps', 'two', 'identified ', 'paper ', 'propose', 'method', 'parsing', 'video', 'semantic', 'steps', 'unsupervised', 'way ', 'proposed', 'method', 'capable', 'providing', 'semantic', ' storyline ', 'video', 'composed', 'objective', 'steps ', 'accomplish', 'using', 'visual', 'language', 'cues', 'joint', 'generative', 'model ', 'proposed', 'method', 'also', 'provide', 'textual', 'description', 'identified', 'semantic', 'steps', 'video', 'segments ', 'evaluate', 'method', 'large', 'number', 'complex', 'youtube', 'videos', 'show', 'results', 'unprecedented', 'quality', 'intricate', 'impactful', 'problem ']


In [ ]:
ABS.build_vocab(train_data.Abstract,train_data.Title,max_size=40000,min_freq=2)
TITLE.build_vocab)
TITLE.vocab= ABS.vocab

In [ ]:
len(TITLE.vocab)

40004

In [ ]:
ABS.vocab.freqs['of']

9616

In [ ]:
assert(TITLE.vocab.stoi ==  ABS.vocab.stoi) #check if both share the same vocab or not

#### Create Iterator

In [ ]:
BATCH_SIZE =64
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE, sort_key = lambda x: len(x.Abstract), sort_within_batch = True,
    device = device)

In [ ]:
batch = next(iter(train_iterator))
batch.Title

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  76,   76, 7886,  ...,   78,   49,  988],
        [2608, 2213,  409,  ..., 2812,  845,  605],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

#### Baseline Model


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, num_layers, dropout): 
        super().__init__()   
        self.hid_dim = hid_dim
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)  
        self.lstm = nn.LSTM(emb_dim, hid_dim, num_layers, dropout = dropout)    
        self.dropout = nn.Dropout(dropout)       
    def forward(self, input_idx):
        #print(input_idx)
        embedded = self.dropout(self.embedding(input_idx))  
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = seq_len,batch_size,embed_dim
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #outputs are always from the top hidden layer
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
  def __init__(self,input_dim,emb_dim,hid_dim,num_layers,dropout,output_dim):
    super().__init__()
    self.hid_dim = hid_dim
    self.num_layers=num_layers
    self.embedding = nn.Embedding(input_dim,emb_dim)
    self.lstm = nn.LSTM(emb_dim,hid_dim,num_layers,dropout=dropout)
    self.dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(hid_dim,output_dim) 
    self.output_dim=output_dim

  def forward(self,input_idx,context_vector,cell_state):
    #input_idx = [batch_size]
    input_idx = input_idx.unsqueeze(0) #Adding a dimenstion at the the first = 1 = seq_len as we are sending word by word
    embedded = self.dropout(self.embedding(input_idx))
    #embedded = [1,batch_size,embed_size]
    #print(embedded.size())
    #print("G")
    outputs, (hidden, cell) = self.lstm(embedded,(context_vector,cell_state))
    #outputs_size = (1,batch_size,hidden_dim)
    outputs = outputs.reshape(-1,self.hid_dim)
    #outputs_size=(batch_size,hid_dim)
    prediction = self.fc(outputs)
    #prediction_size = (batch_size,out_dim)
    return prediction,hidden,cell


In [ ]:
class Seq2Seq(nn.Module): #Combining the encoder and decoder
  def __init__(self,encoder,decoder,device):
    super().__init__()
    self.encoder=encoder
    self.decoder=decoder
    self.device =  device
  def forward(self,input_batches,output_batches,tfr=0.5):
    #input_batches dimension - (seq_len,batch_size)
    #output_batches_dimension - (seq_len,batch_size)
    batch_size = input_batches.shape[1]
    title_len = output_batches.shape[0]
    title_vocab_size = self.decoder.output_dim
    predictions = torch.zeros(title_len, batch_size, title_vocab_size).to(device)
    #print(input_batches.size())
    hidden_state, cell_state = self.encoder(input_batches) 
    #hidden_state/cell_state dimension = num_layers,batch_size,hidden_dim

    x = output_batches[0,:] # Trigger token <SOS>

    for i in range(1, title_len):
      pred, hidden_state, cell_state = self.decoder(x, hidden_state, cell_state)
      #pred = [batch_size,output_dim(vocab_size)]
      predictions[i] = pred
      best_guess = pred.argmax(1) 
      x = output_batches[i,:] if random.random() < tfr else best_guess
    return predictions  

In [ ]:
INPUT_DIM = len(ABS.vocab)
OUTPUT_DIM = len(TITLE.vocab)
ENC_EMB_DIM = 100
DEC_EMB_DIM = 100
HID_DIM = 512
N_LAYERS = 3
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(INPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT,OUTPUT_DIM)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.1, 0.1)
        
model.apply(init_weights)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(40004, 100)
    (lstm): LSTM(100, 512, num_layers=4, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(40004, 100)
    (lstm): LSTM(100, 512, num_layers=4, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
    (fc): Linear(in_features=512, out_features=40004, bias=True)
  )
)

In [ ]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
PAD_IDX = TITLE.vocab.stoi[TITLE.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i,batch in enumerate(iterator):
        
        abstract = batch.Abstract
        title = batch.Title
        #abstract,title = [seq_len,batch_size]
        optimizer.zero_grad()
        predictions = model(abstract, title,0.9)
        #predictions = [seq_len_title,batch_size,title_vocab]
        output_dim = predictions.shape[-1]
        
        predictions = predictions[1:].view(-1, output_dim)#ignoring the first value is the <sos> token
        title = title[1:].view(-1)
        
        loss = criterion(predictions, title)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def test(model, iterator, criterion):    
    model.eval() 
    epoch_loss = 0 
    with torch.no_grad():   
        for i, batch in enumerate(iterator):
          abstract = batch.Abstract
          title = batch.Title
          #abstract,title = [seq_len,batch_size]
          predictions = model(abstract, title,0)
          #predictions = [seq_len_title,batch_size,title_vocab]
          output_dim = predictions.shape[-1]
          predictions = predictions[1:].view(-1, output_dim)#ignoring the first value is the <sos> token
          title = title[1:].view(-1)
          loss = criterion(predictions, title)  
          epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
#to generate title for one abstract
def translate(model,abs,max_length):
  #abs = A single abstract to generate title, data_type = str

  #Preprocessing as done during training
  abs=abs.lower()
  abs = tokenize(abs)
  abs = cleanup_text(abs)
  #convert to a list of idx corresponding to abstract vocab
  num_abs = []
  num_abs.append(ABS.vocab.stoi['<sos>'])
  for w in abs:
    if w in ABS.vocab.stoi:
      num_abs.append(ABS.vocab.stoi[w])
    else:
      num_abs.append(ABS.vocab.stoi['<unk'])
  num_abs.append(ABS.vocab.stoi['<eos>'])
  #covert to Long Tensor
  num_abs = torch.LongTensor(num_abs)
  #add batch_size = 1
  num_abs = num_abs.unsqueeze(1)
  #load them to device
  num_abs = num_abs.to(device)
  #print(num_abs.size())
  #model = model.to(device)

  #model.eval()
  #Forward pass through the encoder
  with torch.no_grad():
    hidden_state, cell_state = model.encoder(num_abs)
  
  #Starts with <sos>
  pred = [ABS.vocab.stoi["<sos>"]]
  for _ in range(max_length):
        previous_word = torch.LongTensor([pred[-1]]).to(device)
        #print(previous_word.size())
        with torch.no_grad():
            output, hidden_state, cell_state = model.decoder(previous_word, hidden_state, cell_state)
            best_guess = output.argmax(1).item()

        pred.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == ABS.vocab.stoi["<eos>"]:
            break

  translated_sentence = [ABS.vocab.itos[idx] for idx in pred]
  return translated_sentence[1:]

In [ ]:
model.parameters

<bound method Module.parameters of Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(61763, 256)
    (lstm): LSTM(256, 600, num_layers=4, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(61763, 256)
    (lstm): LSTM(256, 600, num_layers=4, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
    (fc): Linear(in_features=600, out_features=61763, bias=True)
  )
)>

In [ ]:
demo_sentence = "We propose an architecture for VQA which utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent attention units offers a rich joint embedding of visual and\ntextual features and enables the model to reason relations between several\nparts of the image and question. Our single model outperforms the first place\nwinner on the VQA 1.0 dataset, performs within margin to the current\nstate-of-the-art ensemble model. We also experiment with replacing attention\nmechanisms in other state-of-the-art models with our implementation and show\nincreased accuracy. In both cases, our recurrent attention mechanism improves\nperformance in tasks requiring sequential or relational reasoning on the VQA\ndataset"


In [ ]:
N_EPOCHS = 150
CLIP = 1
import time
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    test_loss = test(model,valid_iterator,criterion)
    end_time = time.time()
    
    #epoch_mins, epoch_secs = epoch_time(start_time, end_time)
      
    #print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTest Loss: {test_loss:.3f}')
    print("After epoch {} , generated title is {}".format(epoch,translate(model,demo_sentence,10)))
 



	Train Loss: 6.752
	Test Loss: 6.508
After epoch 0 , generated title is ['<unk>', 'of', 'for', 'for', 'for', '<eos>']
	Train Loss: 6.356
	Test Loss: 6.656
After epoch 1 , generated title is ['<unk>', 'learning', 'of', '<unk>', 'of', '<unk>', '<eos>']
	Train Loss: 6.071
	Test Loss: 6.914
After epoch 2 , generated title is ['<unk>', 'a', '<unk>', 'approach', 'for', 'the', 'and', '<unk>', '<eos>']
	Train Loss: 5.895
	Test Loss: 7.114
After epoch 3 , generated title is ['<unk>', 'a', '<unk>', 'approach', 'for', 'the', '<unk>', '<eos>']
	Train Loss: 5.747
	Test Loss: 6.993
After epoch 4 , generated title is ['<unk>', 'a', '<unk>', 'approach', 'for', 'the', '<unk>', 'of', 'the', '<eos>']
	Train Loss: 5.618
	Test Loss: 7.009
After epoch 5 , generated title is ['<unk>', 'a', '<unk>', 'approach', 'for', 'the', '<unk>', '<eos>']
	Train Loss: 5.461
	Test Loss: 7.093
After epoch 6 , generated title is ['<unk>', 'a', 'deep', 'convolutional', 'neural', 'network', 'for', 'image', 'classification', '<

In [ ]:
original_title = "Dual Recurrent Attention Units for Visual Question Answering"
generated_title = ['<unk>', 'visual', 'question', 'answering', 'with', 'visual', 'attention', '<eos>']
print("Abstract")
print(demo_sentence)
print("Original Title :",original_title)
print("Generated Title (after ignoring <unk> and <eos> symbols) : ", " ".join(generated_title[1:-1]))

Abstract
We propose an architecture for VQA which utilizes recurrent layers to
generate visual and textual attention. The memory characteristic of the
proposed recurrent attention units offers a rich joint embedding of visual and
textual features and enables the model to reason relations between several
parts of the image and question. Our single model outperforms the first place
winner on the VQA 1.0 dataset, performs within margin to the current
state-of-the-art ensemble model. We also experiment with replacing attention
mechanisms in other state-of-the-art models with our implementation and show
increased accuracy. In both cases, our recurrent attention mechanism improves
performance in tasks requiring sequential or relational reasoning on the VQA
dataset
Original Title : Dual Recurrent Attention Units for Visual Question Answering
Generated Title (after ignoring <unk> and <eos> symbols) :  visual question answering with visual attention
